In [2]:

import os
import numpy as np
import preprocessing as pp
import file_tool as ft

path = r'D:\20240412'
path_list = ft.find_file(path,'.csv')

filename = [os.path.basename(path_list[i])[:-4] for i in range(len(path_list))]####保留文件名

#################default_iter##############
aim_peak = []
scores = 1
#####ctrl ku  ctrl kc

################################内标用到的峰#######################################
wl,spec = ft.read_ava(path_list[0])
maxspec = np.max(spec,axis = 1)
int_peak = [742.361,744.322,746.852]##氢线656.302      氮线742.361,744.322,746.852     氧777.257
peak_para = pp.Peak_integrate(wl,maxspec,int_peak)


for path_num,paths in enumerate(path_list):
    method = 'total_intensity'#'total_intensity' 'internal_standard'
    print('正在计算：' + paths,'method = ' + method)
    wl,spec = ft.read_ava(paths)
    rsd = []

    channel = 0###########10381最后一个通道，8284倒数第二个spec[8284:,:],进行切片###########

    spec = pp.Anomalous_spectrum_removal(spec[channel:,:])
    wl = wl[channel:]
    spec_num = len(spec[0,:])
    


    for wave in range(len(spec[:,0])):    
        rsd.append(pp.RSD_calculate(spec[wave,:],spec_num))
        



    #cun nm spec

    for i in range(spec_num):#calculate slope
        
        spec[:,i] = pp.Normalization(spec[:,i],method,peak_para) 
        
#########################slope 计算部分#########################################  
        single_spec = spec[:,i]
        slope = np.int64((np.append(single_spec,single_spec[-1]) - np.insert(single_spec,0,single_spec[0]))>= 0)#eg  0123 vs 0 -1 1 1 0 ,-1 is the slope between num0&1
        peak = np.insert(slope,0,slope[0]) - np.append(slope,slope[-1])
        if i == 0:
            slope_map = slope
            peak_map = peak
        else:
            slope_map = np.vstack((slope_map,slope)) 
            peak_map = np.vstack((peak_map,peak)) 
            #map 1 = + while 0 = -
    
    slope_map[slope_map == 0] = - 1
    peak_map = peak_map[:,1:-1].T###计算peak形状

    for single_ in slope_map:###计算 arise 和 decay 形状
        for i in range(len(single_)-1):
            if single_[i] != single_[i+1]:
                single_[i] = 0

    scores =  (np.abs(np.sum(peak_map,axis = 1)) +  np.abs(np.sum(slope_map[:,:-1],axis = 0)))/spec_num

###################################################################################
    for i in range(spec_num):
        spec[:,i] = pp.Normalization(spec[:,i],method,peak_para) ###shifou gui yihua
###########cun#####
    wei_spec = (spec.T * scores).T
    wei_mean_spec = np.hstack((filename[path_num],np.mean(wei_spec,axis = 1))).reshape(-1)
    mean_spec = np.hstack((filename[path_num],np.mean(spec,axis = 1))).reshape(-1)
    rsd = np.hstack((filename[path_num],rsd)).reshape(-1)

    if path_num == 0:
        rsd_list = rsd
        mean_list = mean_spec
        wei_mean_list = wei_mean_spec
    else:
        rsd_list = np.vstack((rsd_list,rsd))
        mean_list = np.vstack((mean_list,mean_spec))
        wei_mean_list = np.vstack((wei_mean_list,wei_mean_spec))

    np.savetxt(path + '//' +filename[path_num]+ 'nm.csv',np.hstack((wl,spec)),delimiter=',',fmt = '%s')
    np.savetxt(path + '//' +filename[path_num]+ 'wei.csv',np.hstack((wl,wei_spec)),delimiter=',',fmt = '%s')

wl = np.hstack(('wl',wl.reshape(-1))).reshape(-1)

np.savetxt(path + '//' + path[-4:] + 'rsd.csv',np.vstack((np.array([wl]),rsd_list)).T,delimiter=',',fmt = '%s')
np.savetxt(path + '//' + path[-4:] + 'mean.csv',np.vstack((np.array([wl]),mean_list)).T,delimiter=',',fmt = '%s')
np.savetxt(path + '//' + path[-4:] + 'wei_mean.csv',np.vstack((np.array([wl]),wei_mean_list)).T,delimiter=',',fmt = '%s')

正在计算：D:\20240412\10\10.csv method = total_intensity
正在计算：D:\20240412\11\11.csv method = total_intensity
正在计算：D:\20240412\12\12.csv method = total_intensity
正在计算：D:\20240412\13\13.csv method = total_intensity
正在计算：D:\20240412\14\14.csv method = total_intensity
正在计算：D:\20240412\15\15.csv method = total_intensity
正在计算：D:\20240412\16\16.csv method = total_intensity
正在计算：D:\20240412\17\17.csv method = total_intensity
正在计算：D:\20240412\18\18.csv method = total_intensity
正在计算：D:\20240412\19\19.csv method = total_intensity
正在计算：D:\20240412\20\20.csv method = total_intensity
正在计算：D:\20240412\6\6.csv method = total_intensity
正在计算：D:\20240412\7\7.csv method = total_intensity
正在计算：D:\20240412\8\8.csv method = total_intensity
正在计算：D:\20240412\9\9.csv method = total_intensity
